### Open .pb frozen graf in Tensorboard

In [2]:
import tensorflow as tf
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename ='model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='log/'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

Instructions for updating:
Use tf.gfile.GFile.


### Run on cmd/terminal
`tensorboard --logdir=log/ --port=8080`

### Do a Transform Graph to Optimize model for inference

In [9]:
from tensorflow.python.tools import optimize_for_inference_lib
from tensorflow.tools.graph_transforms import TransformGraph

with tf.gfile.FastGFile('model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    graph_def = optimize_for_inference_lib.optimize_for_inference(graph_def, ['Input'], ['Sigmoid_0004'], tf.float32.as_datatype_enum)
    graph_def = TransformGraph(graph_def, ['Input'], ['Sigmoid_0004'], ['fold_constants', 'sort_by_execution_order'])
    with tf.gfile.FastGFile('fused_graph.pb', 'wb') as f:
            f.write(graph_def.SerializeToString())

In [8]:
help(optimize_for_inference_lib.optimize_for_inference)

Help on function optimize_for_inference in module tensorflow.python.tools.optimize_for_inference_lib:

optimize_for_inference(input_graph_def, input_node_names, output_node_names, placeholder_type_enum, toco_compatible=False)
    Applies a series of inference optimizations on the input graph.
    
    Args:
      input_graph_def: A GraphDef containing a training model.
      input_node_names: A list of names of the nodes that are fed inputs during
        inference.
      output_node_names: A list of names of the nodes that produce the final
        results.
      placeholder_type_enum: The AttrValue enum for the placeholder data type, or
          a list that specifies one value per input node name.
      toco_compatible: Boolean, if True, only runs optimizations that result in
        TOCO compatible graph operations (default=False).
    
    Returns:
      An optimized version of the input graph.



### Check graf again in Tensorboard after TRansform Graph

In [10]:
with tf.Session() as sess:
    model_filename ='fused_graph.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='log_after/'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

- Close tensorboard sebelumnya yang berjalan di terminal
- Run on cmd/terminal untuk log = log_after/ <br>
`tensorboard --logdir=log/ --port=8080`